In [7]:
import os
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import io

In [8]:
# Full Drive access scope
SCOPES = ['https://www.googleapis.com/auth/drive']

# Authenticate and create the Drive service
def authenticate():
    creds = None
    if os.path.exists('../token.json'):
        creds = Credentials.from_authorized_user_file('../token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                '../credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('../token.json', 'w') as token:
            token.write(creds.to_json())
    return build('drive', 'v3', credentials=creds)

def upload_file(file_path, file_name):
    service = authenticate()
    file_metadata = {'name': file_name}
    media = MediaFileUpload(file_path, resumable=True)
    uploaded_file = service.files().create(
        body=file_metadata, media_body=media, fields='id').execute()
    print(f"File uploaded successfully. File ID: {uploaded_file.get('id')}")
    return uploaded_file.get('id')

# Example usage
file_id = upload_file('covertype.csv', 'covertype.csv')

File uploaded successfully. File ID: 15YqRiq5ZBL6lk63baOnFDEvYRTwVGCtr


In [9]:
def download_file(file_id, destination_path):
    service = authenticate()
    request = service.files().get_media(fileId=file_id)

    with io.FileIO(destination_path, 'wb') as file:
        downloader = MediaIoBaseDownload(file, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}%.")


# Example usage: Download the uploaded file
download_file("15YqRiq5ZBL6lk63baOnFDEvYRTwVGCtr", 'downloaded_myfile.csv')

Download 100%.
